In [ ]:
import numpy as np 
import pandas as pd 
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import hstack, csr_matrix
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import re
import gc
import matplotlib.pyplot as plt


# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
train = pd.read_csv("train.tsv", low_memory=False, sep='\t')#, nrows=100000)
test = pd.read_csv("test_stg2.tsv", low_memory=False, sep='\t')#, nrows=100000)
train.info(memory_usage="deep")

In [ ]:
test.info(memory_usage="deep")

# データの確認

In [ ]:
train.head()

train_id / test _id – ユーザー投稿のID  
name – 投稿のタイトル。タイトルに価格に関する情報がある場合（例：$20）はメルカリが事前に削除をして[rm]と置き換えています。  
item_condition_id – ユーザーが指定した商品の状態  
category_name – 投稿カテゴリー  
brand_name – ブランドの名前  
price – 訓練データのみ。実際に売られた価格。米ドル表示。今回のチャレンジの予測ターゲットとなります。  
shipping – 送料のフラグ。「1」は販売者負担。「0」は購入者負担  　
item_description – ユーザーが投稿した商品説明の全文。タイトルと同様に価格情報がある場合は[rm]と置き換えられている  

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)


# trainの基本統計量を表示

In [ ]:
display_all(train.describe(include='all').transpose())

# trainのカテゴリ名、商品説明、投稿タイトル、ブランド名のデータタイプを「category」へ変換する

In [ ]:
train.category_name = train.category_name.astype('category')
train.item_description = train.item_description.astype('category')
train.name = train.name.astype('category')
train.brand_name = train.brand_name.astype('category')

# testのカテゴリ名、商品説明、投稿タイトル、ブランド名のデータタイプを「category」へ変換する

In [ ]:
test.category_name = test.category_name.astype('category')
test.item_description = test.item_description.astype('category')
test.name = test.name.astype('category')
test.brand_name = test.brand_name.astype('category')

# dtypesでデータ形式を確認

In [ ]:
train.dtypes, test.dtypes

# 特徴のある値を確認する

In [ ]:
train.apply(lambda x: x.nunique())

In [ ]:
test.apply(lambda x: x.nunique())

# trainの欠損データの個数と%を確認

In [ ]:
train.isnull().sum(),train.isnull().sum()/train.shape[0]

# testの欠損データの個数と%を確認

In [ ]:
test.isnull().sum(),test.isnull().sum()/test.shape[0]


# trainとtestのidカラム名を変更する

In [ ]:
train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'test_id':'id'})

# 両方のセットへ「is_train」のカラムを追加 

1 = trainのデータ、0 = testデータ

In [ ]:
train['is_train'] = 1
test['is_train'] = 0

# trainのprice(価格）以外のデータをtestと連結

In [ ]:
train_test_combine = pd.concat([train.drop(['price'], axis=1),test],axis=0)

# データの確認

In [ ]:
train_test_combine.head()

# train_test_combineの文字列のデータタイプを「category」へ変換

In [ ]:
train_test_combine.category_name = train_test_combine.category_name.astype('category')
train_test_combine.item_description = train_test_combine.item_description.astype('category')
train_test_combine.name = train_test_combine.name.astype('category')
train_test_combine.brand_name = train_test_combine.brand_name.astype('category')

# combinedDataの文字列を「.cat.codes」で数値へ変換する

In [ ]:
train_test_combine.name = train_test_combine.name.cat.codes
train_test_combine.category_name = train_test_combine.category_name.cat.codes
train_test_combine.brand_name = train_test_combine.brand_name.cat.codes
train_test_combine.item_description = train_test_combine.item_description.cat.codes

# データの中身とデータ形式を表示して確認する

In [ ]:
train_test_combine.dtypes


In [ ]:
train_test_combine.head()

# 「is_train」のフラグでcombineからtestとtrainへ切り分ける

In [ ]:
df_test = train_test_combine.loc[train_test_combine['is_train'] == 0]
df_train = train_test_combine.loc[train_test_combine['is_train'] == 1]

# 「is_train」をtrainとtestのデータフレームから落とす

In [ ]:
df_test = df_test.drop(['is_train'], axis=1)
df_train = df_train.drop(['is_train'], axis=1)

# サイズの確認

In [ ]:
df_test.shape

In [ ]:
df_train.shape

# df_trainへprice（価格）を戻す

In [ ]:
df_train['price'] = train.price

# price（価格）をlog関数で処理

In [ ]:
df_train['price'] = df_train['price'].apply(lambda x: np.log(x) if x>0 else x)

# df_trainを表示して確認

In [ ]:
df_train.head()

# ランダムフォレストモデルの作成

x ＝ price以外の全ての値、y = price（ターゲット）で切り分ける

In [ ]:
x_train, y_train = df_train.drop(['price'], axis=1), df_train.price

モデルの作成

In [ ]:
m = RandomForestRegressor(n_jobs=-1, min_samples_leaf=5, n_estimators=200)
m.fit(x_train, y_train)

スコアを表示

In [ ]:
m.score(x_train, y_train)

# 作成したランダムフォレストのモデル「m」に「df_test」を入れて予測する

In [ ]:
preds = m.predict(df_test)

# 予測値 predsをnp.exp()で処理

In [ ]:
np.exp(preds)

# Numpy配列からpandasシリーズへ変換

In [ ]:
preds = pd.Series(np.exp(preds))

# テストデータのIDと予測値を連結

In [ ]:
submit = pd.concat([df_test.id, preds], axis=1)

# カラム名をメルカリの提出指定の名前をつける

In [ ]:
submit.columns = ['test_id', 'price']

# 提出ファイルとしてCSVへ書き出し

In [ ]:
submit.to_csv('submission.csv', index=False)